In [1]:
import io
import logging
import time
from pathlib import Path

import librosa
import numpy as np
import soundfile
import IPython.display as ipd
from inference import infer_tool
from inference import slicer
from inference.infer_tool import Svc

logging.getLogger('numba').setLevel(logging.WARNING)
chunks_dict = infer_tool.read_temp("inference/chunks_temp.json")

model_path = "logs/32k/G_440000.pth"
config_path = "configs/config.json"
svc_model = Svc(model_path, config_path)
infer_tool.mkdir(["raw", "results"])

C:\Users\Administrator\AppData\Local\Programs\Python\Python310\lib\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (5.1.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


DEBUG:matplotlib.pyplot:Loaded backend agg version v2.2.
INFO:root:Loaded checkpoint 'logs/32k/G_440000.pth' (iteration 1376)


In [2]:
# 支持多个wav文件，放在raw文件夹下
clean_names = ["录音"]
trans = [2]  # 音高调整，支持正负（半音）
spk_list = ['speaker0']  # 每次同时合成多语者音色
slice_db = -40  # 默认-40，嘈杂的音频可以-30，干声保留呼吸可以-50
wav_format = 'flac'  # 音频输出格式

infer_tool.fill_a_to_b(trans, clean_names)
for clean_name, tran in zip(clean_names, trans):
    raw_audio_path = f"raw/{clean_name}"
    if "." not in raw_audio_path:
        raw_audio_path += ".wav"
    infer_tool.format_wav(raw_audio_path)
    wav_path = Path(raw_audio_path).with_suffix('.wav')
    audio, sr = librosa.load(wav_path, mono=True, sr=None)
    wav_hash = infer_tool.get_md5(audio)
    if wav_hash in chunks_dict.keys():
        print("load chunks from temp")
        chunks = chunks_dict[wav_hash]["chunks"]
    else:
        chunks = slicer.cut(wav_path, db_thresh=slice_db)
    print(chunks)
    chunks_dict[wav_hash] = {"chunks": chunks, "time": int(time.time())}
    infer_tool.write_temp("inference/chunks_temp.json", chunks_dict)
    audio_data, audio_sr = slicer.chunks2audio(wav_path, chunks)

    for spk in spk_list:
        audio = []
        for (slice_tag, data) in audio_data:
            print(f'#=====segment start, {round(len(data) / audio_sr, 3)}s======')
            length = int(np.ceil(len(data) / audio_sr * svc_model.target_sample))
            raw_path = io.BytesIO()
            soundfile.write(raw_path, data, audio_sr, format="wav")
            raw_path.seek(0)
            if slice_tag:
                print('jump empty segment')
                _audio = np.zeros(length)
            else:
                out_audio, out_sr = svc_model.infer(spk, tran, raw_path)
                _audio = out_audio.cpu().numpy()
            audio.extend(list(_audio))

        res_path = f'./results/{clean_name}_{tran}key_{spk}.{wav_format}'
        soundfile.write(res_path, audio, svc_model.target_sample, format=wav_format)
ipd.display(ipd.Audio(audio, rate=audio_sr, normalize=False))

load chunks from temp
{'0': {'slice': True, 'split_time': '0,12205'}, '1': {'slice': False, 'split_time': '12205,260096'}}
#=====segment start, 0.254s======
jump empty segment
#=====segment start, 5.164s======
hubert use time:6.472944259643555
vits use time:1.026125431060791


In [23]:
import os

# 文件夹路径
folder_path = './output_audio'

# 遍历文件夹中的所有文件
for file in os.listdir(folder_path):
    file_path = os.path.join(folder_path, file)
    if os.path.isfile(file_path):  # 只处理文件，不处理文件夹
        # 获取文件名和扩展名
        file_name, file_ext = os.path.splitext(file)
        # 构造新的文件名
        new_file_name = file_name + '1' + file_ext
        new_file_path = os.path.join(folder_path, new_file_name)
        # 使用 os 库的 rename() 函数重命名文件
        os.rename(file_path, new_file_path)